In [5]:
from datetime import date

import nest_asyncio
from dotenv import load_dotenv
from pydantic_ai import RunContext, Tool

from agentgenius.agents import BaseAgent
from agentgenius.tools import ToolSet

from agentgenius.core import AgentGENius
from agentgenius.config import config

nest_asyncio.apply()
load_dotenv()

True

In [6]:
agent = AgentGENius.from_file(config.agents_path / "tool_agent.json")


@agent.tool_plain
def ask_user_tool(question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


result = agent.run_sync("is today my birthday?")
print(agent.to_json())
print(result.data)

{"name":"tool_agent","model":"openai:gpt-4o","system_prompt":"You are an expert python programmer. You will create a function that will resolve my question and return an essential value. First, search for a tool that can resolve the question by calling the tool 'read_tools_list', if you found matching one, use it with 'run_tool_from_file'. If no tool is available, make a new one and run with 'run_tool_form_code'. The function that you will create will be directly called. Do not include any example code in your response. Do not include call of the function. Try to make function simple and generic for later use. You always respond with clean, annotated python code, ready to be executed. the code should be self contained and should not use any external libraries except for installed ones (use tool 'available_packages_tool'). All necessary imports should be included locally inside the function, example: def get_time(): import datetime; return datetime.datetime.now(). Make sure the function

In [7]:
def test_tool(value: int) -> str:
    return str(value)


tools = ToolSet(test_tool)

tool = tools.get("test_tool")
assert tool(10) == "10"

assert "test_tool2" not in tools
assert "test_tool" in tools
assert test_tool in tools

tools.remove("test_tool")
tool = tools.get("test_tool")
assert tool is None

In [8]:
def ask_user_tool(ctx: RunContext[str], question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


tools = ToolSet(ask_user_tool)

agent = AgentGENius(
    name="assistant", model="openai:gpt-4o-mini", system_prompt="You are a helpful assistant.", toolset=tools
)


@agent.system_prompt
def add_the_date() -> str:
    return f"The date is {date.today()}."


print(f"{agent.to_json()=}")

j = agent.to_json()
agent2 = BaseAgent.from_json(j)


print(f"{agent2.to_json()=}")
## Agents tolsets aren't the same, not working in the notebook for some reason

agent.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":["ask_user_tool"]}'
agent2.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":[]}'


In [9]:
result = agent.run_sync("is today my birthday?")
print(result.data)

Your birthday is February 5th. Since today is December 29th, it is not your birthday. Your next birthday will be on February 5th, 2025.
